Уже пятый раз заполняю эту форму.
Было бы намного удобней, если бы форма сохранялась для дальнейшего редактирования. Иначе внезапное (или плановое) закрытие браузера приводит к тому, что приходится все заполнять с начала.

А отвечая на вопрос чем заинтересовала возможность стать ревьюером, если кратко: в моем понимании, это хорошая возможность расширять свой кругозор, тренировать хард и софт скилы.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("movie_metadata.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   director_name              4939 non-null   object 
 2   num_critic_for_reviews     4993 non-null   float64
 3   duration                   5028 non-null   float64
 4   director_facebook_likes    4939 non-null   float64
 5   actor_3_facebook_likes     5020 non-null   float64
 6   actor_2_name               5030 non-null   object 
 7   actor_1_facebook_likes     5036 non-null   float64
 8   gross                      4159 non-null   float64
 9   genres                     5043 non-null   object 
 10  actor_1_name               5036 non-null   object 
 11  movie_title                5043 non-null   object 
 12  num_voted_users            5043 non-null   int64  
 13  cast_total_facebook_likes  5043 non-null   int64

In [4]:
df.duration.value_counts().sort_index().sum()

5028

Сколько пропущенных значений содержится в колонке duration?

In [5]:
df.shape[0]-df.duration.value_counts().sort_index().sum()

15

Замените пропущенные значения в колонке duration медианным значением этой колонки.

Какое среднее арифметическое по длительности фильма? Ответ дайте числом с плавающей запятой с округлением до 2 знаков после запятой включительно.


In [6]:
round((df
 .fillna(df.duration.median())
 .duration.describe()
 .get("mean")
),2)

107.19

Cоздайте колонку movie_duration_category, которая содержала бы 3 категории в зависимости от длительности фильма:

Категория "1. < 90", если фильм длится менее 90 минут;
Категория "2. 90-120", если фильм длится от 90 минут до 2 часов (включительно);
Категория "3. > 120", если фильм длится больше 2 часов.
Постройте сводную таблицу для фильмов с годом выпуска после 2000 (включительно), которая отражала бы количество фильмов:

В строках таблицы - год;
В столбцах таблицы - категория длительности фильма ("< 90", "90-120", "> 120");
Год фильма должен быть представлен в формате YYYY
Сколько фильмов с длительностью от 90 минут до 2 часов было выпущено в 2008 году?

In [7]:
df_1 = df.assign(movie_duration_category="1. < 90")
df_1.loc[df_1.duration >= 90, "movie_duration_category"] = "2. 90-120"
df_1.loc[df_1.duration > 120, "movie_duration_category"] = "3. > 120"
(df_1
 .loc[df_1.title_year==2008]
 .movie_duration_category.value_counts()
 ).loc["2. 90-120"]

160

Колонка plot_keywords содержит ключевые слова, характеризующие сюжет фильма. На основе данных этой колонки создайте колонку movie_plot_category, которая содержала бы 4 категории в зависимости от ключевых слов в колонке:

- Категория "love_and_death", если ключевые слова содержат и ключевое слово "love" и ключевое слово "death";
- Категория "love", если ключевые слова содержат ключевое слово "love";
- Категория "death", если ключевые слова содержат ключевое слово "death";
- Категория "other", если ключевые слова не соответствуют перечисленным выше условиям.
Обратите внимание на разделитель ключевых слов. Подумайте о том как правильно определять наличие ключевого слова в списке ключевых слов.

В колонке imdb_score указан пользовательский рейтинг фильма. Постройте таблицу, которая отражала бы средний рейтинг фильмов в зависимости от принадлежности к категории movie_plot_category.

Какой средний рейтинг у фильмов категории "love"? Ответ дайте числом с плавающей запятой с округлением до 2 знаков после запятой включительно.

In [8]:
def love_criteria(df):
    criteria = df[1]
    for clmn in df.columns:
        criteria = (criteria) | df[clmn]
    return criteria

new_df = df.assign(movie_plot_category = "other")
new_df.loc[
     (lambda x: x
      .plot_keywords.str.lower().str.split(r"[|\s]", expand=True)
      .isin(["love"])
      .pipe(love_criteria)
     ),
    "movie_plot_category" 
 ] = "love"
new_df.loc[
     (lambda x: x
      .plot_keywords.str.lower().str.split(r"[|\s]", expand=True)
      .isin(["death"])
      .pipe(love_criteria)
     ),
    "movie_plot_category" 
 ] = "death"
condition = (new_df
      .plot_keywords.str.lower().str.split(r"[|\s]", expand=True)
      .isin(["death"])
      .pipe(love_criteria)
     ) & (new_df
      .plot_keywords.str.lower().str.split(r"[|\s]", expand=True)
      .isin(["love"])
      .pipe(love_criteria)
     )
new_df.movie_plot_category.where(~(condition), other="love_and_death", inplace=True)
print(new_df.groupby("movie_plot_category").agg({"imdb_score": ["mean", "count", "sum"]}))

round(new_df.groupby("movie_plot_category").agg({"imdb_score": "mean"}).loc["love", "imdb_score"], 2)

                    imdb_score               
                          mean count      sum
movie_plot_category                          
death                 6.519886   176   1147.5
love                  6.588987   227   1495.7
love_and_death        6.672727    11     73.4
other                 6.431432  4629  29771.1


6.59

Колонка budget - бюджет фильма. Какой медианный бюджет всех представленных фильмов? Ответ дайте целым числом.

In [9]:
df.budget.describe()

count     5043
unique     440
top       0.0$
freq       492
Name: budget, dtype: object

In [10]:
df.budget.str.split("$", expand=True)[0].astype("float").sort_values().median()


15000000.0

## Вторая часть

## 1. 
Загрузите датасет event_data.csv, который содержит данные использования мобильного приложения пользователей, зарегистрировавшихся в период с 29 июля по 1 сентября 2019 года:

user_id - идентификатор пользователя;
event_date - время совершения события;
event_type - тип события: registration - регистрация в приложении; simple_event - событие клика в приложении; purchase - событие покупки внутри приложения; purchase_amount - величина покупки.
Выделите когорты пользователей на основе недели регистрации в приложении. Идентификатором когорты должен быть порядковый номер недели (например, неделя с 29 июля по 4 августа должна иметь идентификатор 31).

In [83]:
df2 = (pd.read_csv("event_data.csv")
       .astype({
           'event_type': 'category'
       })
       .assign(event_date = lambda x: pd.to_datetime(x.event_date, dayfirst=True))
       .assign(week = lambda x: x.event_date.dt.isocalendar()['week'])
      )

### 1. Какое количество уникальных пользователей в когорте с идентификатором 33?

In [85]:
len(df2.loc[
    (df2.week==33) & (df2.event_type=='registration')
].user_id.unique())

2045

## 2.
Для каждого события выделите показатель lifetime — понедельное время жизни когорты. Показатель lifetime рассчитывают на основе порядкового номера недели, в которую совершено событие, относительно недели регистрации. Например, событие, совершённое 3 августа пользователем из когорты зарегистрировавшихся в 31 неделю, будет совершено в нулевую неделю lifetime, а событие, совершенное этим же пользователем 5 августа — в первую неделю lifetime).

Постройте сводную таблицу изменения показателя Retention Rate по когортам в зависимости от lifetime.

Какой Retention Rate на 3 неделю lifetime у когорты с идентификатором 32? Отсчет недель ведется с 0. Ответ дайте в процентах с округлением до 2 знаков после запятой включительно. Retention Rate представляет собой отношение числа активных пользователей когорты на определенную неделю к общему числу пользователей в когорте. Активным пользователем считается любой пользователь, совершивший событие.

In [202]:
registration_events_df = df2.loc[
            df2.event_type=="registration"
        ]

In [203]:
simple_events_df = (
    df2.loc[
        df2.event_type=="simple_event"
    ]
    .merge(
        registration_events_df
        , how='left'
        , on=['user_id']
        , suffixes=['_se', '_rg']
        , indicator=True
    )
    .assign(lifetime = lambda x: x.week_se - x.week_rg)
)
simple_events_df

,user_id,event_date_se,event_type_se,purchase_amount_se,week_se,event_date_rg,event_type_rg,purchase_amount_rg,week_rg,_merge,lifetime
0,af679d,2019-07-29 02:25:19,simple_event,NaN,31,2019-07-29 00:30:46,registration,NaN,31,both,0
1,71ac11,2019-07-29 06:19:19,simple_event,NaN,31,2019-07-29 00:28:51,registration,NaN,31,both,0
2,85367a,2019-07-29 06:37:26,simple_event,NaN,31,2019-07-29 01:55:28,registration,NaN,31,both,0
3,9b887b,2019-07-29 06:39:29,simple_event,NaN,31,2019-07-29 03:14:53,registration,NaN,31,both,0
4,b2e16e,2019-07-29 06:51:51,simple_event,NaN,31,2019-07-29 01:14:16,registration,NaN,31,both,0
...,...,...,...,...,...,...,...,...,...,...,...
63534,930c23,2019-09-01 23:57:41,simple_event,NaN,35,2019-08-07 15:58:32,registration,NaN,32,both,3
63535,a84999,2019-09-01 23:57:50,simple_event,NaN,35,2019-08-14 05:08:15,registration,NaN,33,both,2
63536,175e4d,2019-09-01 23:59:40,simple_event,NaN,35,2019-08-10 00:31:21,registration,NaN,32,both,3
63537,1c2210,2019-09-01 23:59:51,simple_event,NaN,35,2019-08-17 09:45:33,registration,NaN,33,both,2


In [106]:
retention_df = (simple_events_df
 .groupby(['week_rg', 'lifetime'])
 .agg(
     retention=pd.NamedAgg(column='user_id', aggfunc='count')
 )
 .reset_index()
).merge(
    (simple_events_df
     .groupby(['week_rg'])
     .agg(
         base=pd.NamedAgg(column='user_id', aggfunc='count'))
     .reset_index()
    ), how='left', on=['week_rg']
).assign(retention_rate=lambda x: x.retention / x.base)
retention_df

,week_rg,lifetime,retention,base,retention_rate
0,31,0,3467,17457,0.198602
1,31,1,6743,17457,0.386263
2,31,2,4271,17457,0.244658
3,31,3,2204,17457,0.126253
4,31,4,772,17457,0.044223
5,32,0,3395,16678,0.203562
6,32,1,6745,16678,0.404425
7,32,2,4303,16678,0.258005
8,32,3,2235,16678,0.134009
9,33,0,3657,15420,0.237160


In [138]:
retention_df.pivot(
    index='week_rg',
    columns='lifetime',
    values='retention_rate').fillna(0)

lifetime,0,1,2,3,4
week_rg,,,,,
31,0.198602,0.386263,0.244658,0.126253,0.044223
32,0.203562,0.404425,0.258005,0.134009,0.000000
33,0.237160,0.460506,0.302335,0.000000,0.000000
34,0.335942,0.664058,0.000000,0.000000,0.000000
35,1.000000,0.000000,0.000000,0.000000,0.000000


### 2. Какой Retention Rate на 3 неделю lifetime у когорты с идентификатором 32? 

In [126]:
round(retention_df.loc[
    lambda x: (x.week_rg==32) & (x.lifetime==3)
    , "retention_rate"
].values[0], 2)

0.13

## Третья часть
Постройте сводную таблицу изменения показателя ARPPU (Average Revenue Per Paying User) по когортам в зависимости от lifetime. ARPPU для заданой недели рассчитывается как сумма величин покупок, деленная на количество пользователей, совершивших покупку.

Какой ARPPU на 3 неделю lifetime у когорты с идентификатором 31? Отсчет недель ведется с 0. Ответ дайте числом с плавающей запятой с округлением до 2 знаков после запятой включительно.

In [140]:
purcahse_df = (
    df2.loc[
        df2.event_type=="purchase"
    ]
    .merge(
        df2.loc[
            df2.event_type=="registration"
        ], how='left', on=['user_id'], suffixes=['_pc', '_rg'], indicator=True
    )
    .assign(lifetime = lambda x: x.week_pc - x.week_rg)
)
purcahse_df

,user_id,event_date_pc,event_type_pc,purchase_amount_pc,week_pc,event_date_rg,event_type_rg,purchase_amount_rg,week_rg,_merge,lifetime
0,51dc02,2019-07-29 11:43:11,purchase,10.0,31,2019-07-29 07:16:36,registration,NaN,31,both,0
1,f1c95f,2019-07-29 11:48:46,purchase,50.0,31,2019-07-29 02:35:11,registration,NaN,31,both,0
2,a13a2b,2019-07-29 14:50:07,purchase,10.0,31,2019-07-29 12:32:03,registration,NaN,31,both,0
3,deccc0,2019-07-29 23:38:20,purchase,50.0,31,2019-07-29 13:25:00,registration,NaN,31,both,0
4,11e04e,2019-07-30 03:08:15,purchase,50.0,31,2019-07-29 21:56:29,registration,NaN,31,both,0
...,...,...,...,...,...,...,...,...,...,...,...
6202,7f27ee,2019-09-01 23:31:37,purchase,10.0,35,2019-08-11 14:44:59,registration,NaN,32,both,3
6203,e1d4ac,2019-09-01 23:39:00,purchase,50.0,35,2019-08-13 13:22:58,registration,NaN,33,both,2
6204,f1b677,2019-09-01 23:39:49,purchase,50.0,35,2019-08-13 01:46:17,registration,NaN,33,both,2
6205,0a5860,2019-09-01 23:54:50,purchase,30.0,35,2019-08-28 01:13:29,registration,NaN,35,both,0


In [145]:
arppu_df = (purcahse_df
 .groupby(['week_rg', 'lifetime'])
 .agg(
     revenue=pd.NamedAgg(column='purchase_amount_pc', aggfunc='sum'),
     users=pd.NamedAgg(column="user_id", aggfunc="count")    
 )
 .reset_index()
 .assign(arppu=lambda x: x.revenue / x.users)
)
arppu_df

,week_rg,lifetime,revenue,users,arppu
0,31,0,8890.0,316,28.132911
1,31,1,20540.0,696,29.511494
2,31,2,12210.0,400,30.525000
3,31,3,6120.0,210,29.142857
4,31,4,2010.0,66,30.454545
5,32,0,10850.0,359,30.222841
6,32,1,21050.0,707,29.773692
7,32,2,12600.0,421,29.928741
8,32,3,6260.0,212,29.528302
9,33,0,9790.0,319,30.689655


In [146]:
arppu_df.pivot(
    index="week_rg",
    columns="lifetime",
    values="arppu"
).fillna(0)

lifetime,0,1,2,3,4
week_rg,,,,,
31,28.132911,29.511494,30.525000,29.142857,30.454545
32,30.222841,29.773692,29.928741,29.528302,0.000000
33,30.689655,30.519663,29.171975,0.000000,0.000000
34,30.776699,30.929134,0.000000,0.000000,0.000000
35,30.721925,0.000000,0.000000,0.000000,0.000000


### 3. Какой ARPPU на 3 неделю lifetime у когорты с идентификатором 31? Отсчет недель ведется с 0. Ответ дайте числом с плавающей запятой с округлением до 2 знаков после запятой включительно.

In [148]:
round(arppu_df.loc[
    lambda x: 
    (x.week_rg==31) & (x.lifetime==3)
    , "arppu"
].values[0], 2)

29.14

 ## 4. 
 Временем регистрации будем считать время совершения первого события пользователем. Определите для каждого пользователя время регистрации и время первой покупки. Какое медианное время проходит между регистрацией пользователя и первой покупкой? Ответ дайте в секундах целым числом

In [172]:
first_simple_event_df = (simple_events_df
 .groupby(['user_id'])
 .agg(
     reg_date=pd.NamedAgg(column='event_date_se', aggfunc='min')
 )
 .reset_index()
)
first_purchase_df = (purcahse_df
 .groupby(['user_id'])
 .agg(
     purchase_date=pd.NamedAgg(column='event_date_pc', aggfunc='min')
 )
 .reset_index()
)

In [173]:
first_purchase_df

,user_id,purchase_date
0,00049f,2019-08-18 17:31:56
1,00082c,2019-08-14 12:57:41
2,001d5d,2019-08-26 13:03:22
3,002195,2019-08-18 22:35:43
4,0025c1,2019-08-10 11:37:00
...,...,...
4278,ffb3f3,2019-08-04 06:47:40
4279,ffbe61,2019-08-21 21:14:07
4280,ffc66a,2019-08-16 06:05:05
4281,ffc88b,2019-08-09 15:36:48


In [212]:
(registration_events_df
 .merge(
     first_purchase_df, how='inner', on=['user_id']
 )
 .assign(time_to_purchase=lambda x: (x.purchase_date - x.event_date).dt.seconds)
 ).time_to_purchase.median()

43623.0

## 3.

### 3.1 
Укрупненно, DataFrame - это таблица. В ней есть строки и столбцы. Причем, у каждой строки есть свой номер (индекс). 
А Series - это один столбец, для которого по строкам также есть номер строки.

Например, ниже - DataFrame:
Номер | Имя | Цена | Количество
1          | Дуб  | 10     | 12
2          | Вяз  | 12     | 18
3          | Бук  | 10     | 14

В нем 3 строки и 3 столбца (1-й столбец, номер, является специальным столбцом, называемым "индекс")

А вот это - Series:
Номер | Имя 
1          | Дуб  
2          | Вяз  
3          | Бук 
В нем только 1 столбец, в котором содержится 3 значения по строкам.

### 3.2
В целом логика близка к правильной, однако есть несколько важных моментов, которые нужно учесть, чтобы получить верный результат:

1. Что произойдет в текущем варианте решения, когда нужный жанр впервые встретится в таблице? 

В цикле будут перебираться последовательно все имена. Как только имя из таблицы будет совпадать с искомым именем, увеличится переменная сount на единицу. То есть, сount будет равным 1.
Здесь пока все корректно.

Далее, в этом же условии, стоит инструкция "return count". То есть, "заверши выполнение функции и верни count". Здесь функция остановится и вернет 1. Все, дальше цикл исполняться не будет. А это не совсем корректно - среди оставшихся значений в списке могут быть нужные нам.

2. Что произойдет, если нужного нам имени нет в списке?
Допустим, список жанров выглядит вот так: ['pop', 'hip-hop', 'rap' ]. И мы ищем имя 'hip'.
- На первом шаге 'pop' равно 'hip'? Нет. Переходим на след.шаг
- На втором 'hip-hop' равно 'hip'? Нет. Переходим на след.шаг
- На третьем 'rap' равно 'hip'? Нет. Список закончился, выходим из цикла.

Далее идет инструкция print(count) - "распечатай на экран значение в переменной count". То есть, выведи на экран 0.

И здесь же третий важный момент: 

3. что "вернет" функция find_genre в этом случае? До инструкции return мы не дошли. В случае, когда программист не указал явным образом, что возвращать, программа сама вернет из функции специальное значение None - по сути, это "ничего, ничто, пусто".

Поэтому, чтобы получить нужный результат, надо доработать код:
- сделать так, чтобы функция не останавливалась на первом же нахождении имени в списке
- возвращать из функции в любом случае результат поиска